In [10]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import requests
import json
import tls_client
from unidecode import unidecode
import time

In [15]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

requests = tls_client.Session(
    client_identifier="chrome112",
)

response1 = requests.get('https://api.prizepicks.com/projections')

prizepicks = response1.json()

In [16]:
prizepicks

{'data': [{'type': 'projection',
   'id': '1598175',
   'attributes': {'board_time': '2023-09-29T19:48:14-04:00',
    'custom_image': None,
    'description': 'NYJ/WAS',
    'end_time': None,
    'flash_sale_line_score': None,
    'is_promo': False,
    'line_score': 152.5,
    'projection_type': 'Single Stat',
    'rank': 0,
    'refundable': True,
    'start_time': '2023-10-01T13:00:00-04:00',
    'stat_type': 'Rush+Rec Yds (Combo)',
    'status': 'pre_game',
    'tv_channel': None,
    'updated_at': '2023-09-29T20:51:47-04:00'},
   'relationships': {'duration': {'data': None},
    'league': {'data': {'type': 'league', 'id': '9'}},
    'new_player': {'data': {'type': 'new_player', 'id': '161664'}},
    'projection_type': {'data': {'type': 'projection_type', 'id': '2'}},
    'stat_type': {'data': {'type': 'stat_type', 'id': '466'}}}},
  {'type': 'projection',
   'id': '1603805',
   'attributes': {'board_time': '2023-09-29T23:00:00-04:00',
    'custom_image': None,
    'description': '

In [13]:
df = pd.json_normalize(response1,record_path='data')
df

NotImplementedError: 

#original
import urllib3
import json
import urllib
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

pp_props_url = 'https://api.prizepicks.com/projections'
opener = AppURLopener()
response = opener.open(pp_props_url)
http = urllib3.PoolManager()
myheaders = {'Content-Type': 'application/json'}
mydata = {'username': '***', 'password': '****'}
print(response.status)
#data = json.loads(response.read())
#df = pd.json_normalize(data,record_path='data')
#df

In [85]:
df2 = pd.json_normalize(json_data,record_path='included')
df2.head(3)

,id,type,attributes.lfg_ignored_leagues,attributes.name,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.position,attributes.team,attributes.team_name,relationships.league.data.id,relationships.league.data.type,attributes.active,attributes.icon,attributes.last_five_games_enabled,attributes.projections_count,attributes.rank,relationships.projection_filters.data
0,232,stat_type,[],Hits Allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141671,new_player,NaN,Luan Lacerda\t,https://i.imgur.com/fYgDxPy.png,MMA,12.0,None,F,Luan Lacerda\t,None,12,league,NaN,NaN,NaN,NaN,NaN,NaN
2,41619,new_player,NaN,Matheus Rossetto,https://s3-us-west-2.amazonaws.com/static.fant...,SOCCER,82.0,None,M,Atlanta,None,82,league,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df2.drop(df2.columns[df2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [87]:
#getting names dict from df2
display(df2.head())
names = df2[['type','id', 'attributes.name']]
names = names.loc[names['type']=='new_player']
names = names[['id','attributes.name']]

name = names['attributes.name'].tolist()
id = names['id'].tolist()

names_dict = {id[i]: name[i] for i in range(len(name))}

names_dict

,id,type,attributes.lfg_ignored_leagues,attributes.name,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.position,attributes.team,attributes.team_name,relationships.league.data.id,relationships.league.data.type,attributes.active,attributes.icon,attributes.last_five_games_enabled,attributes.projections_count,attributes.rank,relationships.projection_filters.data
0,232,stat_type,[],Hits Allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141671,new_player,NaN,Luan Lacerda\t,https://i.imgur.com/fYgDxPy.png,MMA,12.0,None,F,Luan Lacerda\t,None,12,league,NaN,NaN,NaN,NaN,NaN,NaN
2,41619,new_player,NaN,Matheus Rossetto,https://s3-us-west-2.amazonaws.com/static.fant...,SOCCER,82.0,None,M,Atlanta,None,82,league,NaN,NaN,NaN,NaN,NaN,NaN
3,67698,new_player,NaN,Steven Kwan,https://static.prizepicks.com/images/players/m...,MLB,2.0,Cleveland,OF,CLE,Guardians,2,league,NaN,NaN,NaN,NaN,NaN,NaN
4,64444,new_player,NaN,koalanoob,https://i.imgur.com/dzrRKkM.png,VAL,159.0,None,G,M80,None,159,league,NaN,NaN,NaN,NaN,NaN,NaN


{'141671': 'Luan Lacerda\t',
 '41619': 'Matheus Rossetto',
 '67698': 'Steven Kwan',
 '64444': 'koalanoob',
 '78978': 'Adonis Frias',
 '44529': 'Fidel Ambríz',
 '64781': 'Billy Horschel',
 '127610': 'Kevin Newman',
 '15148': 'NiKo',
 '1666': 'Emiliano Grillo',
 '69529': 'Yoán Moncada',
 '142009': 'Caliste',
 '16556': 'krii',
 '140386': 'Bijan Robinson',
 '46893': 'Óliver Torres',
 '132098': 'Mark Stone',
 '127583': 'Andrew McCutchen',
 '16044': 'SHOCK',
 '67931': 'Alex Wood',
 '41354': 'Darlington Nagbe',
 '142049': 'Facundo González',
 '38588': 'Devonta Smith',
 '64797': 'Cameron Young',
 '10778': 'Duncan Robinson',
 '67524': "Ke'Bryan Hayes",
 '41628': 'Miles Robinson',
 '41617': 'Santiago Sosa',
 '64473': 'Georgios Giakoumakis',
 '65009': 'Jonathan Lewis',
 '142048': 'Sebastián Boselli',
 '127938': 'Luis García',
 '15155': 'flusha',
 '41602': 'Malte Amundsen',
 '132051': 'Sam Bennett',
 '16372': 'Esteban Ocon',
 '141404': 'Nikola Jokic + Aaron Gordon',
 '94214': 'Dendi',
 '55281': 'J

In [88]:
df.head()

,id,type,attributes.board_time,attributes.custom_image,attributes.description,attributes.end_time,attributes.flash_sale_line_score,attributes.is_promo,attributes.line_score,attributes.projection_type,...,relationships.league.data.type,relationships.new_player.data.id,relationships.new_player.data.type,relationships.projection_type.data.id,relationships.projection_type.data.type,relationships.stat_type.data.id,relationships.stat_type.data.type,attributes.discount_percentage,relationships.duration.data.id,relationships.duration.data.type
0,1279426,projection,2023-04-23T00:00:00-04:00,None,2023 ATL,None,NaN,False,1075.50,Single Stat,...,league,140386,new_player,2,projection_type,15,stat_type,NaN,NaN,NaN
1,1340331,projection,2023-05-23T22:30:00-04:00,None,MIA,2023-06-01T20:30:00-04:00,0.5,True,27.50,Single Stat,...,league,1291,new_player,2,projection_type,19,stat_type,98.0,NaN,NaN
2,1275535,projection,2023-04-27T20:00:00-04:00,None,2023 CAR,None,NaN,False,3649.50,Single Stat,...,league,140303,new_player,2,projection_type,16,stat_type,NaN,NaN,NaN
3,1355338,projection,2023-01-05T19:50:00-05:00,None,Philipe Lins,None,NaN,False,14.75,Single Stat,...,league,16661,new_player,2,projection_type,43,stat_type,NaN,NaN,NaN
4,1356194,projection,2023-05-25T20:08:00-04:00,None,VGK (Games 1-4),None,NaN,False,7.50,Single Stat,...,league,132052,new_player,2,projection_type,30,stat_type,NaN,NaN,NaN


In [89]:
#viewing all the columns to see what i need

base = df[:-1]
base.loc[15]

id                                                           1354697
type                                                      projection
attributes.board_time                      2023-05-30T00:36:00-04:00
attributes.custom_image                                         None
attributes.description                                           MIA
attributes.end_time                                             None
attributes.flash_sale_line_score                                 NaN
attributes.is_promo                                            False
attributes.line_score                                           13.5
attributes.projection_type                               Single Stat
attributes.rank                                                    1
attributes.refundable                                           True
attributes.start_time                      2023-06-01T20:40:00-04:00
attributes.stat_type                                        Rebounds
attributes.status                 

In [90]:
id[0:5]

['141671', '41619', '67698', '64444', '78978']

In [91]:
ids = df['relationships.new_player.data.id'].unique()
print(len(ids))


731


In [92]:
#trimming excess columns and adding actual player names

df = base[['attributes.description','attributes.line_score',
        'attributes.stat_type','relationships.new_player.data.id','relationships.league.data.id','attributes.start_time']]

df['Player'] = df['relationships.new_player.data.id'].map(names_dict)
df = df[['Player','attributes.description', 'attributes.line_score', 'attributes.stat_type','relationships.league.data.id','attributes.start_time']]

df = df.rename(columns={"attributes.description": "Team", "attributes.line_score": "Line", 
                        'attributes.stat_type':'Stat','relationships.league.data.id':'League',
                       'attributes.start_time':'Date'})

C:\Users\User\AppData\Local\Temp\ipykernel_256\2042246692.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player'] = df['relationships.new_player.data.id'].map(names_dict)


In [93]:
df.League.unique()

array(['163', '7', '12', '251', '250', '2', '84', '1', '125', '159',
       '231', '82', '6', '5', '192', '121', '174', '124', '8'],
      dtype=object)

In [94]:
full = df.copy(deep=True)

In [95]:
#df = df.loc[df.League=='7']
df['Date'] = df['Date'].apply(lambda x: x.split('T')[0])
df['Player'] = df['Player'].apply(lambda x: unidecode(x).replace('_',' '))
df

,Player,Team,Line,Stat,League,Date
0,Bijan Robinson,2023 ATL,1075.50,Rush Yards,163,2023-09-07
1,Nikola Jokic,MIA,27.50,Points,7,2023-06-01
2,Bryce Young,2023 CAR,3649.50,Pass Yards,163,2023-09-07
3,Maxim Grishin,Philipe Lins,14.75,Fight Time (Mins),12,2023-06-03
4,Aaron Ekblad,VGK (Games 1-4),7.50,Shots On Goal,251,2023-06-03
...,...,...,...,...,...,...
1458,William Karlsson,FLA,0.50,Hits,8,2023-06-03
1459,Eetu Luostarinen,VGK,2.50,Hits,8,2023-06-03
1460,William Carrier,FLA,3.00,Hits,8,2023-06-03
1461,Radko Gudas,VGK,4.50,Hits,8,2023-06-03


In [96]:
syntax = {
    'Points': 'PTS',
    'Rebs': 'TRB',
    'Asts': 'AST',
    'Rebounds': 'TRB',
    '3-PT Made': '3P',
    'Assists': 'AST',
    'Turnovers': 'TOV',
    'Blks': 'BLK',
    'Pts': 'PTS',
    'Pts+Asts': 'PTS+AST',
    'Pts+Rebs+Asts':'PTS+TRB+AST',
    'Pts+Rebs':'PTS+TRB',
    'Rebs+Asts':'TRB+AST',
    'Blks+Stls':'BLK+STL',
    'Stls': 'STL',
    'Free Throws Made': 'FT',
    'Blocked Shots': 'BLK',
    'Steals': 'STL',
    'Personal Fouls': 'PF',
    'FG Attempted': 'FGA'
}
df = df.replace(syntax)
df.Stat.unique()

array(['Rush Yards', 'PTS', 'Pass Yards', 'Fight Time (Mins)',
       'Shots On Goal', 'PTS+AST', 'TRB', 'Hitter Strikeouts',
       'Fantasy Score', 'PTS+TRB+AST', 'Birdies or Better Matchup',
       'Pitcher Strikeouts', 'Strokes', 'Runs', 'Pars Or Better',
       'Fairways Hit', 'Significant Strikes', '1st Pit Stop Time (sec)',
       'MAPS 1-2 Kills', 'Pitcher Fantasy Score', 'Birdies Or Better',
       'Total Bases', 'MAP 3 Kills', 'Pitches Thrown',
       'Greens In Regulation', 'Hits Allowed', 'AST', 'TRB+AST',
       'PTS+TRB', 'Hitter Fantasy Score', 'Earned Runs Allowed',
       'Takedowns', 'BLK+STL', 'TOV', '3P', 'Shots', 'Shots On Target',
       'Clearances', 'Tackles', 'Passes Attempted', 'Goalie Saves',
       'Crosses', 'Shots Assisted', 'Break Points Won', 'Total Games',
       'Aces', 'Hits+Runs+RBIS', 'FT', '1st Inning Runs Allowed',
       'Pitching Outs', 'MAP 1 Kills', 'Walks Allowed', 'Holes Played',
       'Receiving Yards', '1st Inning Hits Allowed', 'MAP 3 He

In [97]:
len(df)

1463

In [98]:
df = df[df.Stat != 'Fantasy Score']

In [99]:
len(df)

1429

In [100]:
df = df.replace('Fred VanVleet\t','Fred VanVleet')
df = df.replace('Nicolas Claxton','Nic Claxton')
df = df.replace('Robert Williams III','Robert Williams')
df = df.replace('LeBron James\t','LeBron James')
df= df.replace('KJ Martin Jr.','Kenyon Martin Jr.')
df= df.replace('Xavier Tillman','Xavier Tillman Sr.')#xav

In [101]:
df = df.reset_index(drop=True)

In [102]:
today= dt.date.today()
df.sample(5)

,Player,Team,Line,Stat,League,Date
377,Denis Bouanga,Leon,1.5,Shots On Target,82,2023-05-31
515,Kyle Lowry,DEN,10.5,PTS+TRB,7,2023-06-01
1317,sjuush,G2,34.5,MAPS 1-2 Kills,124,2023-05-31
771,Stuart Fairchild,BOS,1.5,Hits+Runs+RBIS,2,2023-05-31
676,Bam Adebayo,DEN 1st Half,1.5,FT,84,2023-06-01


In [103]:
f'lines{today.month}_{today.day}'

'lines5_31'

In [104]:
nba = df.loc[df.League=='7']
nba.to_csv(f'Lines/lines{today.month}_{today.day}')

In [105]:
pd.option_context('display.max_columns', 5)
df

,Player,Team,Line,Stat,League,Date
0,Bijan Robinson,2023 ATL,1075.50,Rush Yards,163,2023-09-07
1,Nikola Jokic,MIA,27.50,PTS,7,2023-06-01
2,Bryce Young,2023 CAR,3649.50,Pass Yards,163,2023-09-07
3,Maxim Grishin,Philipe Lins,14.75,Fight Time (Mins),12,2023-06-03
4,Aaron Ekblad,VGK (Games 1-4),7.50,Shots On Goal,251,2023-06-03
...,...,...,...,...,...,...
1424,William Karlsson,FLA,0.50,Hits,8,2023-06-03
1425,Eetu Luostarinen,VGK,2.50,Hits,8,2023-06-03
1426,William Carrier,FLA,3.00,Hits,8,2023-06-03
1427,Radko Gudas,VGK,4.50,Hits,8,2023-06-03


In [106]:
df['Date'].max()

'2023-09-07'

In [107]:
df.to_csv(f'Lines/full_lines{today.month}_{today.day}')

In [108]:
df.League.unique()

array(['163', '7', '12', '251', '250', '2', '84', '1', '125', '159',
       '231', '82', '6', '5', '192', '121', '174', '124', '8'],
      dtype=object)